In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
classes=('penis','vagina','breast','butt','naked-body','naked-bottom','underwear')

In [14]:

## config_file = '../configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
config_file = '../configs/censor/faster_rcnn_x101_32x4d_fpn_1x_censor.py'
config_file = '../configs/censor/faster_rcnn_x101_32x4d_fpn_1x_censor-freeze-s1-coloraug.py'
config_file = '../configs/censor/faster_rcnn_x101_32x4d_fpn_1x_censor-freeze-s4.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
#checkpoint_file = '../checkpoints/faster_rcnn_r50_fpn_1x_20181010-3d1b3351.pth'
checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor/epoch_17.pth'
checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor-rot/epoch_29.pth'
checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor-freeze-s1-coloraug/epoch_25.pth'
checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor-freeze-s1/epoch_20.pth'
#checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor-freeze-s4/epoch_48.pth'
#checkpoint_file = '../work_dirs/faster_rcnn_x101_32x4d_fpn_1x_censor/epoch_20.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

two_stage.py: pretrained=  None


In [4]:
# build the model from a config file and a checkpoint file

lines = open('/home/jovyan/fast-data/tusou_test/script/valid.lst','r').readlines()
PORN6=['hentai','anime','norm','porn','sexy','very_sexy']
dat={}
print(len(lines))
for line in lines:
    a = line.rstrip().split(' ')
    file = a[0]
    annt = PORN6[int(a[1])]
    if annt not in dat.keys():
        dat[annt]=[]
        dat[annt].append(file)
    else:
        dat[annt].append(file)
#print(dat)

11556


In [1]:
# test a single image
import cv2
img = 'demo.jpg'
img = '/home/jovyan/fast-data/tusou_test/porn/(m=bJ4hdaaecaa)(mh=-EDlnUGRnKxMCZ-J)original_208989461.jpg'
for imgfil in dat['porn'][:40]:
    img_bgr = cv2.imread(imgfil)
    #print(imgfil)
    img_rgb=cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    #result = inference_detector(model, img_bgr)
    result = inference_detector(model, imgfil)
    #print(result)
    show_result_pyplot(model, imgfil, result,score_thr=0.3)

NameError: name 'dat' is not defined

In [44]:
print(result[6].shape[0])

0
